<a href="https://colab.research.google.com/github/DeekshithaGona/NLP/blob/main/GetPrescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (1,929 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
import pytesseract
import cv2 as cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
pytesseract.pytesseract.tesseract_cmd=r'/usr/bin/tesseract'


# to extract prescription part

In [3]:
import pytesseract
from PIL import Image

# Load the image
image_path = '/content/med.jpg'
img = Image.open(image_path)
print(img.size)
# Define the coordinates of the ROI  (left, top, right, bottom) (prescription part)
mednames_coordinates = (100, 400, 650,1340 )  # Adjust these values based on your image
# Define the coordinates of the ROI  (left, top, right, bottom) (frequency part)
frequency_coordinates = (650, 400, 800,1340 )  # Adjust these values based on your image

# Crop the image to the ROI
roi_image1 = img.crop(mednames_coordinates)
roi_image2 = img.crop(frequency_coordinates)
# Perform OCR on the cropped image
roi_text1 = pytesseract.image_to_string(roi_image1)
print(roi_text1)
print('hehehehheeheh!!!i did itttt')
roi_text2 = pytesseract.image_to_string(roi_image2)
print(roi_text2)


(1130, 1600)
SALBAIR TRANSHALER INHALER
Levosalbutamol 50mcg

To be inhaled as per instructions found in the instruction booklet
MOXCLAV 625 TABLET
Amoxicillin 500mg + Clavulanic Acid 125mg

MAXITHRAL 500 TABLET
Azithromycin 500mg

MONTEK LC TABLET

MONTELUKAST 10mg + LEVOCETRIZINE 5mg

SOMPRAZ 20 TABLET
Esomeprazole 20mg

hehehehheeheh!!!i did itttt
(MN - AF - EN -1

1-0-0-1
Before food

1-1-0-1
After food

1-0-0-0
After food

0-0-0-1
After food

1-0-0-1
Before food



#  crop and extract relevant info only

In [4]:
import pytesseract
from PIL import Image

def crop_and_ocr(image, coordinates):
    roi_image = image.crop(coordinates)
    roi_text = pytesseract.image_to_string(roi_image)
    return roi_text

def info():
    # Load the image
    image_path = '/content/med.jpg'
    img = Image.open(image_path)
    #print(img.size)

    # Define the coordinates of the ROIs (left, top, right, bottom)
    mednames_coordinates = (100, 400, 650, 1340)
    frequency_coordinates = (650, 400, 800, 1340)

    # Perform OCR on the cropped images
    mednames_text = crop_and_ocr(img, mednames_coordinates)
    frequency_text = crop_and_ocr(img, frequency_coordinates)

    #print("Extracted Medicine Names:")
    print(mednames_text)
    #print("Extracted Frequencies:")
    #print(frequency_text)

if __name__ == "__main__":
    info()


SALBAIR TRANSHALER INHALER
Levosalbutamol 50mcg

To be inhaled as per instructions found in the instruction booklet
MOXCLAV 625 TABLET
Amoxicillin 500mg + Clavulanic Acid 125mg

MAXITHRAL 500 TABLET
Azithromycin 500mg

MONTEK LC TABLET

MONTELUKAST 10mg + LEVOCETRIZINE 5mg

SOMPRAZ 20 TABLET
Esomeprazole 20mg



In [5]:
import re
def extract_medicine_names_and_frequencies(text):

    image_path = '/content/med.jpg'
    img = Image.open(image_path)
    mednames_text = crop_and_ocr(img, mednames_coordinates)  # Replace with your extracted medicine names text
    frequency_text = crop_and_ocr(img, frequency_coordinates)
    lines = text.split('\n')
    #medicine_name_pattern = r'^[A-Z][A-Z ]*(?:TABLET|INHALER|SYRUP|CAPSULE)'
    medicine_name_pattern = r'^[A-Z0-9 ]+$'

    frequency_pattern = r'^(\d-\d-\d-\d)+$'

    medicine_names = []
    frequencies = []
    current_medicine = None

    for line in lines:
        # Check for medicine name
        if re.match(medicine_name_pattern, line):
            current_medicine = line
            medicine_names.append(current_medicine)
        # Check for frequency
        elif re.match(frequency_pattern, line):
            frequencies.append(line)

    return medicine_names, frequencies


In [6]:
mednames_text = crop_and_ocr(img, mednames_coordinates)
medicine_names, _ = extract_medicine_names_and_frequencies(mednames_text)
print("Number of Medicine Names:", len(medicine_names))
print(medicine_names)

frequency_text = crop_and_ocr(img, frequency_coordinates)
_, frequencies = extract_medicine_names_and_frequencies(frequency_text)
print("Number of Frequencies:", len(frequencies))
print(frequencies)


Number of Medicine Names: 5
['SALBAIR TRANSHALER INHALER', 'MOXCLAV 625 TABLET', 'MAXITHRAL 500 TABLET', 'MONTEK LC TABLET', 'SOMPRAZ 20 TABLET']
Number of Frequencies: 5
['1-0-0-1', '1-1-0-1', '1-0-0-0', '0-0-0-1', '1-0-0-1']


In [10]:
def final_info():
    medicine_frequency_dict = dict(zip(medicine_names, frequencies))
    for medicine, frequency in medicine_frequency_dict.items():
        print(f"Medicine: {medicine}, Frequency: {frequency}")
    return medicine_frequency_dict
final_info()

Medicine: SALBAIR TRANSHALER INHALER, Frequency: 1-0-0-1
Medicine: MOXCLAV 625 TABLET, Frequency: 1-1-0-1
Medicine: MAXITHRAL 500 TABLET, Frequency: 1-0-0-0
Medicine: MONTEK LC TABLET, Frequency: 0-0-0-1
Medicine: SOMPRAZ 20 TABLET, Frequency: 1-0-0-1


{'SALBAIR TRANSHALER INHALER': '1-0-0-1',
 'MOXCLAV 625 TABLET': '1-1-0-1',
 'MAXITHRAL 500 TABLET': '1-0-0-0',
 'MONTEK LC TABLET': '0-0-0-1',
 'SOMPRAZ 20 TABLET': '1-0-0-1'}